In [1]:
import os
import sys
import json
sys.path.append('../')

from pathlib import Path
from rocket_rag.node_indexing import *
from llama_index.readers.file import PyMuPDFReader

c:\Users\HAOXUAN\miniconda3\envs\agent\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


In [2]:
cfg = json.load(open("../config/configs.json"))

INSTANCES_DIR = '../data/instances/'
INFERENCE_DIR = '../data/inference/'
STATES = ['normal', 
            'backlash1', 'backlash2',
            'lackLubrication1', 'lackLubrication2',
            'spalling1', 'spalling2', 'spalling3', 'spalling4', 'spalling5', 'spalling6', 'spalling7', 'spalling8']
LOADS= ['20kg', '40kg', '-40kg']
DOC_DIR = '../docs'

### Documentation indexing

In [3]:
loader = PyMuPDFReader()
documents = loader.load_data(os.path.join(DOC_DIR, 'data_description.pdf'))

In [14]:
txt_transform = TextTransform(cfg=cfg)
txt_node_indexer = TextNodeIndexer(f'../store/doc_indexing/data_description.pkl')

In [6]:
text_chunks = []
doc_idxs = []
for doc_idx, doc in tqdm(enumerate(documents)):
    cur_text_chunk = txt_transform.split_text(doc.text)
    text_chunks.extend(cur_text_chunk)
    doc_idxs.extend([doc_idx] * len(cur_text_chunk))

0it [00:00, ?it/s]

In [7]:
clean_text_chunks = [chunk for chunk in text_chunks if len(chunk) > 0]
clean_text_chunks

['Data set for "Data-based Detection and Diagnosis of Faults \nin Linear Actuators" \n \n1. Introduction \nThe dataset presented here was acquired for the study of degradation in linear \nactuators, particularly electro-mechanical actuators (EMA). The data was acquired \nfrom an instrumented rig (described in section 2) where a ball-screw actuator moved \nleft to right following a defined motion profile with a level of load selected by the user. Nut position and motor current measurements were acquired during the tests. Initially the rig was tested in normal conditions (absence of faults) under different \nmotion and loading scenarios. Subsequently different mechanical faults, including lack \nof lubrication, spalling and backlash, were seeded in the system. Position and motor \ncurrent data were acquired under these conditions to study monitoring methods to \ndetect faults and degradation in this particular type of systems. This document describes in detail the data files available in

In [9]:
embeds = [txt_transform.get_embedding_from_openai(text_chunk).squeeze().tolist() for text_chunk in clean_text_chunks]

In [12]:
txt_nodes = txt_node_indexer.indexing(clean_text_chunks, embeds)
txt_node_indexer.save_nodes(txt_nodes, '../store/doc_indexing/data_description.pkl')

2024-09-24 16:44:02.778 | DEBUG    | rocket_rag.node_indexing:indexing:130 - Indexing txt nodes...
2024-09-24 16:44:02.779 | DEBUG    | rocket_rag.node_indexing:indexing:139 - Indexing txt nodes DONE!
2024-09-24 16:44:02.780 | DEBUG    | rocket_rag.node_indexing:save_nodes:71 - Creating a new file at ../store/doc_indexing/data_description.pkl...
2024-09-24 16:44:02.781 | DEBUG    | rocket_rag.node_indexing:save_nodes:74 - Saving all nodes...
2024-09-24 16:44:02.803 | DEBUG    | rocket_rag.node_indexing:save_nodes:78 - All nodes are stored.


### Time series indexing

In [15]:
ts_transform = TimeSeriesTransform(cfg=cfg)

In [16]:
load = '20kg'
for load in LOADS:
    loguru.logger.debug(f'{load} time series nodes indexing...')

    load_num = load[:2]
    ids = [os.listdir(os.path.join(INSTANCES_DIR, load, state)) for state in STATES]
    ids = [filename for sublist in ids for filename in sublist]
    
    ts = []
    for f in ids:
        state = re.match(fr'(.*)_{load_num}', f).group(1)
        temp_ts_df = pd.read_csv(os.path.join(INSTANCES_DIR, load, state, f))
        ts.append(ts_transform.smoothing(ts_df=temp_ts_df, field='current'))
    
    ts_node_indexer = TimeSeriesNodeIndexer(ts_transform=ts_transform, nodes_filename=f'../store/ts_indexing/current_nodes_{load}.pkl')
    ts_node_indexer.indexing(ts=ts, ids=ids, meta_info={'load': load})

2024-09-24 16:52:07.214 | DEBUG    | __main__:<module>:3 - 20kg time series nodes indexing...
2024-09-24 16:52:10.141 | DEBUG    | rocket_rag.node_indexing:indexing:172 - Indexing time series nodes...


  0%|          | 0/455 [00:00<?, ?it/s]

2024-09-24 16:52:45.474 | DEBUG    | rocket_rag.node_indexing:indexing:182 - Indexing time series nodes DONE.
2024-09-24 16:52:45.475 | DEBUG    | rocket_rag.node_indexing:save_nodes:74 - Saving all nodes...
2024-09-24 16:52:55.201 | DEBUG    | rocket_rag.node_indexing:save_nodes:78 - All nodes are stored.
2024-09-24 16:52:55.202 | DEBUG    | __main__:<module>:3 - 40kg time series nodes indexing...
2024-09-24 16:52:58.048 | DEBUG    | rocket_rag.node_indexing:indexing:172 - Indexing time series nodes...


  0%|          | 0/452 [00:00<?, ?it/s]

2024-09-24 16:53:33.641 | DEBUG    | rocket_rag.node_indexing:indexing:182 - Indexing time series nodes DONE.
2024-09-24 16:53:33.641 | DEBUG    | rocket_rag.node_indexing:save_nodes:71 - Creating a new file at ../store/ts_indexing/current_nodes_40kg.pkl...
2024-09-24 16:53:33.642 | DEBUG    | rocket_rag.node_indexing:save_nodes:74 - Saving all nodes...
2024-09-24 16:53:43.241 | DEBUG    | rocket_rag.node_indexing:save_nodes:78 - All nodes are stored.
2024-09-24 16:53:43.242 | DEBUG    | __main__:<module>:3 - -40kg time series nodes indexing...
2024-09-24 16:53:46.199 | DEBUG    | rocket_rag.node_indexing:indexing:172 - Indexing time series nodes...


  0%|          | 0/455 [00:00<?, ?it/s]

2024-09-24 16:54:21.722 | DEBUG    | rocket_rag.node_indexing:indexing:182 - Indexing time series nodes DONE.
2024-09-24 16:54:21.723 | DEBUG    | rocket_rag.node_indexing:save_nodes:71 - Creating a new file at ../store/ts_indexing/current_nodes_-40kg.pkl...
2024-09-24 16:54:21.724 | DEBUG    | rocket_rag.node_indexing:save_nodes:74 - Saving all nodes...
2024-09-24 16:54:31.449 | DEBUG    | rocket_rag.node_indexing:save_nodes:78 - All nodes are stored.
